In [174]:
import pandas as pd

# Importing data
dayparts_url = 'https://raw.githubusercontent.com/juanpaul96/holcim_DAE_test/main/dayparts_sample.csv'
dayparts_raw = pd.read_csv(dayparts_url)

In [136]:
########## Data Exploration ##########

#Here I'm checking things like: data types and fixing them, count of rows, checking for duplicates, 
# and I'm doing a QA to check data consistency in sales and quantity.

In [16]:
# Check initial data types
print("Dataset columns type:\n", dayparts_raw.dtypes)

Dataset columns type:
 REST_KEY                     int64
Rest Coop                   object
Reporting Day               object
Daypart Name                object
Daypart Description         object
Daypart Sales $            float64
Daypart Transaction Qty      int64
dtype: object


In [15]:
#count of each column
print(dayparts_raw.count())

REST_KEY                   2450
Rest Coop                  2450
Reporting Day              2450
Daypart Name               2450
Daypart Description        2450
Daypart Sales $            2450
Daypart Transaction Qty    2450
dtype: int64


In [175]:
# Convert 'Reporting Day' to datetime
dayparts_datatypes = dayparts_raw.copy()

dayparts_datatypes['Reporting Day'] = pd.to_datetime(dayparts_datatypes['Reporting Day'], errors='coerce')
dayparts_datatypes['Daypart Sales $'] = pd.to_numeric(dayparts_datatypes['Daypart Sales $'], errors='coerce')
dayparts_datatypes['Daypart Transaction Qty'] = pd.to_numeric(dayparts_datatypes['Daypart Transaction Qty'], errors='coerce')

# Rename columns in dayparts_datatypes to use underscores instead of spaces
dayparts_datatypes.columns = [col.strip().replace(" ", "_") for col in dayparts_datatypes.columns]

/tmp/ipykernel_157/2582607048.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dayparts_datatypes['Reporting Day'] = pd.to_datetime(dayparts_datatypes['Reporting Day'], errors='coerce')


In [54]:
dayparts_datatypes["Reporting_Day"].agg(['min','max'])

min   2010-01-02
max   2011-01-31
Name: Reporting_Day, dtype: datetime64[ns]

In [55]:
print(dayparts_datatypes.dtypes)

REST_KEY                            int64
Rest_Coop                          object
Reporting_Day              datetime64[ns]
Daypart_Name                       object
Daypart_Description                object
Daypart_Sales_$                   float64
Daypart_Transaction_Qty             int64
dtype: object


In [176]:
dayparts_corrected = dayparts_datatypes.copy()

# Convert daypart names to uppercase for consistency
dayparts_corrected['Daypart_Name'] = dayparts_corrected['Daypart_Name'].str.upper()

# Pivot so that dayparts become columns
pivoted = dayparts_corrected.pivot_table(
    index=['Reporting_Day', 'REST_KEY'],
    columns='Daypart_Name',
    values='Daypart_Transaction_Qty',
    aggfunc='sum'
).reset_index()

# Calculate new checks using 'TOTAL DAY' column if it exists
pivoted['Check1_Reg+Break=Total'] = (
    (pivoted.get('REGULAR MENU', 0) + pivoted.get('BREAKFAST', 0)).round(2) == pivoted.get('TOTAL DAY', 0).round(2)
)

pivoted['Check2_M+A+L+D+E=Total'] = (
    (pivoted.get('MORNING', 0) +
     pivoted.get('AFTERNOON', 0) +
     pivoted.get('LUNCH', 0) +
     pivoted.get('DINNER', 0) +
     pivoted.get('EVENING', 0)).round(2) == pivoted.get('TOTAL DAY', 0).round(2)
)

pivoted['Check3_M+A+L+4PM=Total'] = (
    (pivoted.get('MORNING', 0) +
     pivoted.get('AFTERNOON', 0) +
     pivoted.get('LUNCH', 0) +
     pivoted.get('AFTER 4PM', 0)).round(2) == pivoted.get('TOTAL DAY', 0).round(2)
)

pivoted


Daypart_Name,Reporting_Day,REST_KEY,AFTER 4PM,AFTERNOON,BREAKFAST,DINNER,EVENING,LATE NIGHT,LUNCH,MORNING,REGULAR MENU,TOTAL DAY,Check1_Reg+Break=Total,Check2_M+A+L+D+E=Total,Check3_M+A+L+4PM=Total
0,2010-01-02,1364,235,75,180,156,79,31,159,180,469,649,True,True,True
1,2010-01-02,5357,143,80,159,119,24,0,140,159,363,522,True,True,True
2,2010-01-02,13369,178,83,119,150,28,1,117,119,378,497,True,True,True
3,2010-01-02,13604,146,97,122,109,37,0,141,122,384,506,True,True,True
4,2010-01-03,1364,198,83,169,157,41,9,133,169,414,583,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2011-01-30,13369,173,69,85,147,26,0,116,85,358,443,True,True,True
241,2011-01-30,13604,84,94,160,84,0,0,149,160,327,487,True,True,True
242,2011-01-31,1364,244,121,197,210,34,1,160,208,536,733,True,True,True
243,2011-01-31,5357,192,111,330,162,30,3,288,362,623,953,True,True,True


In [177]:
# Create a unique key by concatenating identifying columns
dayparts_datatypes['dup_key'] = (
    dayparts_datatypes['REST_KEY'].astype(str) + '_' +
    dayparts_datatypes['Reporting_Day'].astype(str) + '_' +
    dayparts_datatypes['Daypart_Name']
)

# Check for duplicate rows based on the unique key
dayparts_datatypes['dup_check'] = dayparts_datatypes.duplicated('dup_key', keep=False)

# Check for incorrect data: negative sales or transaction quantity
dayparts_datatypes['invalid_data'] = (
    (dayparts_datatypes['Daypart_Sales_$'] < 0) |
    (dayparts_datatypes['Daypart_Transaction_Qty'] < 0)
)

# Show rows flagged as duplicates or invalid
invalid_or_duplicate_rows = dayparts_datatypes[
    dayparts_datatypes['dup_check'] | dayparts_datatypes['invalid_data']
]
invalid_or_duplicate_rows

,REST_KEY,Rest_Coop,Reporting_Day,Daypart_Name,Daypart_Description,Daypart_Sales_$,Daypart_Transaction_Qty,dup_key,dup_check,invalid_data


In [178]:
dayparts_datatypes.columns = (
    dayparts_datatypes.columns
      .str.strip()
      .str.lower()
      .str.replace("_$", "", regex=False)
)

dayparts_datatypes.drop(columns=["dup_key","dup_check","invalid_data"], inplace=True)

In [ ]:
########## Data Transformation ##########

#I noticed we only have a month for two years. 2010 and 2011. I'm going to split them to check them separetly. Also, I'm checking missing data for eacg

In [179]:
# Display the number of records in each split
dayparts_2010 = dayparts_datatypes[dayparts_datatypes['reporting_day'].dt.year == 2010].copy()
dayparts_2011 = dayparts_datatypes[dayparts_datatypes['reporting_day'].dt.year == 2011].copy()

len_2010 = len(dayparts_2010)
len_2011 = len(dayparts_2011)

(len_2010, len_2011)

(1240, 1210)

In [189]:
dayparts_2010.to_csv('dayparts_2010_silver.csv', index=False)
dayparts_2011.to_csv('dayparts_2011_silver.csv', index=False)